In [1]:
from sklearn.datasets import fetch_california_housing
 
data = fetch_california_housing()
print(data.feature_names)
 
X, y = data.data, data.target

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [2]:
import copy
import numpy as np
import torch
import tqdm
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
### Griffin Model


class CustomNN(nn.Module):
    def __init__(self, input_size, layer_sizes, output_size):
        super(CustomNN, self).__init__()
        layers = []

        # Input layer
        layers.append(nn.Linear(input_size, layer_sizes[0]))
        layers.append(nn.ReLU())

        # Hidden layers
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(nn.ReLU())

        # Output layer
        layers.append(nn.Linear(layer_sizes[-1], output_size))

        # Combine all layers into a sequential model
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Function to create dataloaders
def create_dataloaders(f_scl, o_scl, tf_scl, to_scl, batch_size=32):
    # Convert numpy arrays to PyTorch tensors
    f_scl_tensor = torch.tensor(f_scl, dtype=torch.float32)
    o_scl_tensor = torch.tensor(o_scl, dtype=torch.float32)
    tf_scl_tensor = torch.tensor(tf_scl, dtype=torch.float32)
    to_scl_tensor = torch.tensor(to_scl, dtype=torch.float32)

    # Create TensorDatasets
    train_dataset = torch.utils.data.TensorDataset(f_scl_tensor.unsqueeze(0), o_scl_tensor.unsqueeze(0))
    test_dataset = torch.utils.data.TensorDataset(tf_scl_tensor.unsqueeze(0), to_scl_tensor.unsqueeze(0))

    # Create DataLoaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Function to train the model
def train_model_new(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_test_loss += loss.item()

        avg_test_loss = total_test_loss / len(test_loader)
        test_losses.append(avg_test_loss)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

    return train_losses, test_losses

# Function to plot the losses
def plot_losses(train_losses, test_losses, name):
    epochs = np.arange(1, len(train_losses) + 1)
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Test Loss vs. Epoch')
    plt.legend()
    plt.show()
    plt.savefig("/ocean/projects/ees220005p/gmooers/Investigations/Model_Performance/"+name+",png")


### end Griffin Model